In [204]:
import gizmo_analysis as gizmo
import halo_analysis as halo
import numpy as np
import utilities as ut
from gc_utils import get_halo_tree, open_snapshot, snapshot_name  # noqa: F401
from scipy.optimize import minimize, minimize_scalar

In [205]:
simulation = "m12i"
snapshot = 600

sim_dir = "../../../simulations/"
# sim_dir = "../../simulations/"
data_dir = "data/"
sim_codes = data_dir + "external/simulation_codes.json"

fire_dir = sim_dir + simulation + "/" + simulation + "_res7100/"

In [206]:
halt = get_halo_tree(simulation, sim_dir)

Retrieving Halo Tree.....................: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


In [207]:
halt.keys()

dict_keys(['am.phantom', 'am.progenitor.main', 'axis.b/a', 'axis.c/a', 'catalog.index', 'central.index', 'central.local.index', 'descendant.index', 'descendant.snapshot', 'dindex', 'final.index', 'host.distance', 'host.index', 'host.velocity', 'host.velocity.rad', 'host.velocity.tan', 'major.merger.snapshot', 'mass', 'mass.180m', 'mass.200c', 'mass.200m', 'mass.500c', 'mass.bound', 'mass.lowres', 'mass.vir', 'position', 'position.offset', 'progenitor.co.dindex', 'progenitor.co.index', 'progenitor.last.dindex', 'progenitor.main.index', 'progenitor.main.last.dindex', 'progenitor.number', 'radius', 'scale.radius', 'scale.radius.klypin', 'snapshot', 'spin.bullock', 'spin.peebles', 'tid', 'vel.circ.max', 'vel.std', 'velocity', 'velocity.offset'])

In [208]:
# part = open_snapshot(snapshot, fire_dir)

# lets use cold gas and star particles
# cold gass
# tsel = np.log10(part["gas"]["temperature"]) < 4.5
part = gizmo.io.Read.read_snapshots(["star", "gas"], "index", snapshot, fire_dir, assign_hosts_rotation=True)

In [209]:
def mass_dif(r_test, host_name, gas_tot):
    gas_mask_test = part["gas"].prop(f"{host_name}.distance.principal.total") <= r_test
    gas_mass_test = np.sum(part["gas"]["mass"][gas_mask_test])

    mass_dif = np.abs((gas_tot / 2) - gas_mass_test)
    return mass_dif

In [210]:
host_index = 0
r_vir = halt["radius"][host_index]
host_name = ut.catalog.get_host_name(host_index)

gas_mask = part["gas"].prop(f"{host_name}.distance.principal.total") < r_vir
gas_tot = np.sum(part["gas"]["mass"][gas_mask])

res = minimize_scalar(mass_dif, bounds=(0, r_vir), method="bounded", args=(host_name, gas_tot))
r_50 = res.x

In [211]:
def kappa_co(part, snapshot: int, r_50: float, log_t_max: float = 4.5):
    # get star values ####################################
    # only select stars within r_50
    star_mask = part["star"].prop(f"{host_name}.distance.principal.total") < r_50

    # get 3D positions and velocities
    vel_xyz_star = part["star"].prop(f"{host_name}.velocity.principal")[star_mask]
    pos_xyz_star = part["star"].prop(f"{host_name}.distance.principal")[star_mask]

    # particle distances from z-axis
    star_rho = part["star"].prop(f"{host_name}.distance.principal.cylindrical")[:, 0][star_mask]

    # star mass
    star_mass = part["star"]["mass"][star_mask]

    # total stellar kinematic energy
    ek_tot_star = (star_mass) * 0.5 * np.linalg.norm(vel_xyz_star, axis=1) ** 2

    # get angular momentume and create mask
    lz_star = (pos_xyz_star[:, 0] * vel_xyz_star[:, 1] - pos_xyz_star[:, 1] * vel_xyz_star[:, 0]) * star_mass
    lz_star_mask = lz_star > 0

    # get energies of co-rotating particles
    K_co_star = np.sum(
        0.5
        * star_mass[lz_star_mask]
        * (lz_star[lz_star_mask] / (star_mass[lz_star_mask] * star_rho[lz_star_mask])) ** 2
    )

    # get gas values #####################################
    # only select cold gas particles within r_50
    tsel = np.log10(part["gas"]["temperature"]) < log_t_max
    gas_mask = part["gas"].prop(f"{host_name}.distance.principal.total") < r_50

    # get 3D positions and velocities
    vel_xyz_gas = part["gas"].prop(f"{host_name}.velocity.principal")[gas_mask & tsel]
    pos_xyz_gas = part["gas"].prop(f"{host_name}.distance.principal")[gas_mask & tsel]

    # particle distances from z-axis
    gas_rho = part["gas"].prop(f"{host_name}.distance.principal.cylindrical")[:, 0][gas_mask & tsel]

    # gas mass
    gas_mass = part["gas"]["mass"][gas_mask & tsel]

    # total gas kinematic energy
    ek_tot_gas = (gas_mass) * 0.5 * np.linalg.norm(vel_xyz_gas, axis=1) ** 2

    # combined total energy
    ek_tot = np.sum(ek_tot_star) + np.sum(ek_tot_gas)

    # get angular momentume and create mask
    lz_gas = (pos_xyz_gas[:, 0] * vel_xyz_gas[:, 1] - pos_xyz_gas[:, 1] * vel_xyz_gas[:, 0]) * gas_mass
    lz_gas_mask = lz_gas > 0

    # get energies of co-rotating particles
    K_co_gas = np.sum(
        0.5
        * gas_mass[lz_gas_mask]
        * (lz_gas[lz_gas_mask] / (gas_mass[lz_gas_mask] * gas_rho[lz_gas_mask])) ** 2
    )

    # combine stars and cold gas ###################################
    ek_tot = np.sum(ek_tot_star) + np.sum(ek_tot_gas)
    K_co = K_co_star + K_co_gas

    kappa_co = K_co / ek_tot

    snap_id = snapshot_name(snapshot)

    kappa_dict = {}
    kappa_dict[snap_id] = kappa_co

    return kappa_dict

In [213]:
kappa_co(part, snapshot, r_50)

{'snap600': 0.622937503388397}